In [66]:
from ast import literal_eval
from collections import deque
import time

import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

Part 2 only solvable by Gemini 3 - also in the iteration with Gemini, I decided to change my initial solution for part 1 from a DFS to a BFS, which I could have done by myself for cleaning.

Anyway, part 2 is tricky, since I couldn't solve the problem with a dfs. I haven't used a lienar solver for a long time, so I needed the help of Gemini 3. (Still insisting to use scipy). 
This is my learning experience and hopefully I can remember this kind of problem the next time. So, I can start from this solution and adapt it to a similiar problem. Also, that's why I left the comments from Gemini in the script.

## Day 10

### Test

In [67]:
test = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
"""

test = test.split("\n")[:-1]

In [68]:
def parse_instructions(instructions):

    lights = []
    buttons = []
    joltages = []
    
    for i in instructions:
        light, rest = i.split("] ")
        light = light[1:]
        light = light.replace(".", "0")
        light = light.replace("#", "1")
        lights.append(light)
    
        button, joltage = rest.split(" {")
    
        button = button.split()
        button = [literal_eval(i) for i in button]
        button = [i if isinstance(i,tuple) else tuple([i]) for i in button]
        buttons.append(button)
        
        joltage = joltage[:-1]
        joltage =joltage.split(",")
        joltage = [int(i) for i in joltage]
        joltages.append(joltage)

    return lights, buttons, joltages

In [70]:
def update_current(current, button):

    current_lst = list(current)
    for b in button:
        current_lst[b] = "1" if current_lst[b] == "0" else "0"
    new = "".join(current_lst)
    
    return new
    
    
def shortest_combination(buttons, target):

    start = "0" * len(target)
    
    queue = deque([(start, [])])
    
    visited = set()
    visited.add(start)
    

    while queue:
        current, path = queue.popleft()

        if current == target:
            return path

        if len(path) >= len(target)*2:
            continue

        for i, button in enumerate(buttons):
            
            new = update_current(current, button)

            if new not in visited:
                visited.add(new)

                queue.append((new, path + [i]))

    return None

In [71]:
results = []
lights, buttons, joltages = parse_instructions(test)

for i, j in zip(buttons, lights):

    result = shortest_combination(i, j)

    results.append(len(result))

In [72]:
assert sum(results) == 7

## Part 1

In [73]:
with open("../../../advent_of_code_input/2025/day_10/input.txt", "r") as f:
    data = f.read()

data = data.split("\n")[:-1]

In [74]:
start_time = time.perf_counter()

results = []

lights, buttons, joltages = parse_instructions(data)

for i, j in zip(buttons, lights):

    result = shortest_combination(i, j)

    results.append(len(result))
    
end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

0.09966529160737991 seconds


In [75]:
sum(results)

455

## Part 2

### Test

In [76]:
test = """[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}
"""

test = test.split("\n")[:-1]

In [89]:
def solve_linear(buttons, joltage):
    
    num_vars = len(buttons)      # We want to find x0, x1, x2... (count for each button)
    num_equations = len(joltage)       # One equation per Joltage index

    # 2. BUILD THE MATRIX (A)
    A = np.zeros((num_equations, num_vars))
    
    for col, button in enumerate(buttons):
        for row in button:
            A[row, col] = 1

    # 3. CONSTRAINTS
    # In scipy, we set lower_bound = upper_bound = Target
    constraints = LinearConstraint(A, lb=joltage, ub=joltage)

        # By setting lb and ub to the same list (joltage):
        # 1. The result must be >= joltage
        # 2. The result must be <= joltage
    
    # 4. VARIABLES SETUP
    # Variables must be Integers (1)
    integrality = np.ones(num_vars) 

        # In the scipy.optimize.milp function, the integrality array tells the solver which variables must be whole numbers and which are allowed to be decimals (fractions).
        # 0 = Continuous (Decimals allowed, e.g., 2.5, 0.333)
        # 1 = Integer (Whole numbers only, e.g., 2.0, 0.0, 5.0)
    
    # Variables must be >= 0 (Cannot press a button -1 times)
    bounds = Bounds(lb=0, ub=np.inf)
    
    # 5. OBJECTIVE
    # We want to minimize the TOTAL number of presses.
    # Cost vector c = [1, 1, 1...] (all buttons cost 1 press)
    c = np.ones(num_vars)

        # What would happen if you changed it?
        # If you passed c = [0, 0, 0...] (Zeros):
        # The solver would find any valid solution that satisfies the math. It might return a solution with 1,000 steps because it "costs" nothing to add more steps. It wouldn't guarantee the shortest path.
        # If you passed c = [100, 1, 1...]:
        # You would be telling the solver: "Button 0 costs $100 to press, but others cost $1." The solver would try desperately to avoid using Button 0, even if it meant pressing Button 1 fifty times to compensate.
    
    # 6. SOLVE
    res = milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)

    if res.success:
        # Round to nearest int because solvers use floats internally
        counts = [int(round(x)) for x in res.x]
        
        path = []
        for i, count in enumerate(counts):
            path.extend([i] * count)
        return path
    else:
        return None


In [90]:
start_time = time.perf_counter()

results = []
lights, buttons, joltages = parse_instructions(test)

for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

    result = solve_linear(i, k)

    results.append(len(result))

end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

0.0062645841389894485 seconds


In [91]:
assert sum(results) == 33

## Puzzle

In [86]:
with open("../../../advent_of_code_input/2025/day_10/input.txt", "r") as f:
    data = f.read()

data = data.split("\n")[:-1]

In [87]:
start_time = time.perf_counter()

results = []
lights, buttons, joltages = parse_instructions(data)

for n, (i, _, k) in enumerate(zip(buttons, lights, joltages)):

    result = solve_linear(i, k)

    results.append(len(result))

end_time = time.perf_counter()

elapsed = end_time - start_time
print(elapsed, "seconds")

0.11286320816725492 seconds


In [88]:
sum(results)

16978